<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/high_level_postprocessing/confusion_matrix_activity_pre_sw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
  !mkdir -p local_modules/postprocessing_sw_HL

In [2]:
%%writefile local_modules/postprocessing_sw_HL/__init__.py

import numpy as np
from sklearn.metrics import confusion_matrix

def confusion_matrix_activity_pre_sw(test_label, pred,time,lim_inf_colazione,lim_sup_colazione,lim_inf_pranzo,lim_sup_pranzo,lim_inf_cena, lim_sup_cena):
  label_time = (test_label,time)
  pred_time = (pred,time)

  for j in range(0,3):
    if j == 0: 
      lim_inf = lim_inf_colazione
      lim_sup = lim_sup_colazione
      activity = 'colazione'
    if j == 1: 
      lim_inf = lim_inf_pranzo
      lim_sup = lim_sup_pranzo
      activity = 'pranzo'
    if j == 2: 
      lim_inf = lim_inf_cena
      lim_sup = lim_sup_cena
      activity = 'cena'

    #prendo le label che ci indicano l'attività svolta solo nella finestra di tempo compresa tra lim_inf e lim_sup 
    real= []

    i = 0 
    while i < len(label_time[0]):
      if label_time[1][i]> lim_inf and label_time[1][i]<lim_sup:
        real.append(label_time[0][i])
      i = i+1

    #prendo le predizioni che ci indicano l'attività svolta solo nella finestra di tempo compresa tra lim_inf e lim_sup
    pred_sintesi = []

    i = 0 
    while i < len(pred_time[0]):
      if pred_time[1][i]>lim_inf and pred_time[1][i]<lim_sup:
        pred_sintesi.append(pred_time[0][i])
      i = i+1

    #calcolo la matrice di confusione solo per il pasto indicato dalla variabile activity
    print("\nCompute confusion matrix senza sw " + activity)
    y_true = real
    y_true = np.array(y_true,dtype=np.int)
    n_y_pred = pred_sintesi
    tn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()
    print("TN", tn)
    print("FP", fp)
    print("FN", fn)
    print("TP", tp)
    other = 100*tn/(tn+fp)
    pasto = 100*tp/(fn+tp)
    print("Other corretti: %.2f %%" % other)
    print("Pasto corretti: %.2f %%" % pasto)
    
    



Overwriting local_modules/postprocessing_sw_HL/__init__.py
